<a href="https://colab.research.google.com/github/nneibaue/alumni_shuffler/blob/master/Alumni_Shuffler_Official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Alumni Shuffler</h1>


Hello all! Welcome to Google Colab, this awesome tool for sharing Python notebooks in Google Drive. This has been a fun project to work on, and we hope that it is a useful! We live in a strange time, and are lucky to have technology that allows to stay connected while apart. Not only is it a tool, but this project is an example of two people in different states who have never met in person collaborating on a project together -- a project whose sole purpose is to help facilitate the virtual interaction of people from all over the country.  

<br>

---

<br>

The Alumni Shuffler is a tool that helps intelligently create breakout groups during large Zoom calls. The project is in its infancy right now, and is currently being developed by Nathan Neibauer and Hayden Blair. 

Given a virtual Zoom event with upwards of 20 or 30 people, identified by certain characterics ('track', 'year', 'hair color', 'likes _The Office_ ', etc.), this tool can help ensure that everyone can...

* Interact with everyone else in their group
* Interact with as many different people as possible

...without eating up too much mental real-estate from the coordinator, who likely wants to spend more time interacting with students and less time fussing with a spreadsheet. 

Alumni data must be saved in a spreadsheet somewhere on Google Drive for the user who is operating this notebook with columns representing the categories to sort by. An example might look like this

<br>

Name | track | year | hair_color | hard_working |
---|---|---| --- | ---
Leslie | optics | 2017 | blonde | yes
Ron | polymer | 2014 | brown | no
Jean Ralphio | semi | 2012 | black | no
April | sensors | 2019 | black | yes
Ann | semi | 2013 | brown | yes

<br>

For now, I would imagine that 'track' and 'year' are the only identifiers, but the code should work with any number of them.

### Imports

The following cell clones the github repo so private libraries can be imported.

In [1]:
#@title Clone Github Repo

BRANCH_NAME = "nate_testing" #@param {type:"string"}

import os
import sys
import shutil

ROOT = '/content'
os.chdir(ROOT)
REPO_NAME = 'alumni_shuffler'
REPO_URL = f'https://github.com/nneibaue/{REPO_NAME}'
REPO_PATH = os.path.join(ROOT, REPO_NAME)


# Remove old repo
print('Removing old repo...')
!rm -rf $REPO_PATH

print('Cloning from github...')
!git clone $REPO_URL
os.chdir(REPO_PATH)

if BRANCH_NAME != 'master':
  !git checkout --track origin/$BRANCH_NAME
  !git config user.email "colab_anon@gmail.com"
else:
  !git pull
  
if REPO_PATH not in sys.path:
  print(f'Adding {REPO_PATH} to path')
  sys.path.append(REPO_PATH)

os.chdir(ROOT)

Removing old repo...
Cloning from github...
Cloning into 'alumni_shuffler'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 237 (delta 37), reused 182 (delta 19), pack-reused 0
Receiving objects: 100% (237/237), 7.93 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Branch 'nate_testing' set up to track remote branch 'nate_testing' from 'origin'.
Switched to a new branch 'nate_testing'
Adding /content/alumni_shuffler to path


In [2]:
# Other imports
import html_maker
import zoom_sesh
import ipywidgets 
from google.colab import drive, widgets, output
from IPython.display import display, HTML

# Google drive mounting
drive.mount('/content/gdrive')
DRIVE_BASE=('/content/gdrive/My Drive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#@title Zoom Session Manager

#@markdown Enter full Google Drive path to Google Sheets file containing student
#@markdown info for this meeting:
student_file = "" #@param {type:"string"}